In [1]:
from bs4 import BeautifulSoup as bs
import json
import jsonlines
from tqdm import tqdm

In [2]:
file = open('med.cd.az.xml', 'r', encoding = 'utf-8')
content = file.readlines()
bs_contents = []
for line in tqdm(content):
    bs_contents.append(bs(line, 'xml'))
print(len(bs_contents))

100%|███████████████████████████████████| 44309/44309 [00:21<00:00, 2032.49it/s]

44309


In [3]:
def call_example(EXAMPLES):
    examples_dic = {}
    EXAMPLE = EXAMPLES.find('EXAMPLE')
    if EXAMPLE:
        EXAMPLE = EXAMPLE.get_text()
        examples_dic['EXAMPLE'] = EXAMPLE
    trans = EXAMPLES.find("trans")
    if trans:
        trans = trans.get_text()
        examples_dic['TRANSLATION'] = trans
    return examples_dic

In [11]:
def def_trans_ex(sense):
    sense_dict = {}
    # STYLE-LEVEL 
    if entry.find('STYLE-LEVEL') != None:
        STYLE_LEVEL = entry.find('STYLE-LEVEL').get_text()
        sense_dict['STYLE_LEVEL'] = STYLE_LEVEL
    # DEFINITION
    if sense.find('DEFINITION') != None:
        sense_dict['DEFINITION'] = sense.find('DEFINITION').get_text()
    # TRANSLATION
    if sense.find('trans') != None:
        sense_dict['TRANSLATION'] = sense.find("trans").get_text()
    #EXAMPLES
    if sense.find('EXAMPLES') != None:
        # call function
        examples_dic = call_example(sense.find('EXAMPLES'))
        sense_dict['EXAMPLES'] = examples_dic

    #SUB-SENSE
    if sense.find('SUB-SENSE') != None:

        sub_sense_list = []
        for sense in entry.find_all('SUB-SENSE'):
            
            sub_sense_dic = def_trans_ex(sense)
            
  
            sub_sense_list.append(sub_sense_dic)
        sense_dict['SUB-SENSE'] = sub_sense_list
                        
    return sense_dict

In [17]:

word ={}


for bs_content in tqdm(bs_contents):
    word_dict = {}
    for entry in bs_content.find_all('HOMOGRAPH'):
        # tip1: view what tags each entry may contain before proceeding.
        # tip2: not all entries have all the attributes. BeautifulSoup returns `None` when it finds nothing.
        # Note that some tags might not look exactly like those in the output example 
            # (e.g. SENSE45, TRANSLATION). 
            # So please remember to change the tag names as required when you save them to your own dict.

        id = entry['ID']
        #print(id)
        if entry.find('ENTRY0'):
            vocab = entry.find('ENTRY0')
        else:
            vocab = entry.find('ENTRY13')
        vocab = vocab.get_text()
        
        #print(vocab)
        if entry.find('PART-OF-SPEECH') != None:
            part_of_speech = entry.find('PART-OF-SPEECH')
            part_of_speech = part_of_speech.get_text()
            word_dict['PART-OF-SPEECH'] = part_of_speech
        # GRAMMAR
        if entry.find('GRAMMAR') != None:
            GRAMMAR = entry.find('GRAMMAR')
            GRAMMAR = GRAMMAR.get_text()
            word_dict['GRAMMAR'] = GRAMMAR
        # MENU
        if entry.find('MENU') != None:
            MENU = entry.find('MENU')
            ONE_MENU = entry.find_all('ONE-MENU')
            menu_list = []
            for single_menu in ONE_MENU:
                menu_sense = single_menu.find('MENU-SENSE').get_text()
                #MENU-DATA
                menu_data = single_menu.find('MENU-DATA').get_text()
                #trans
                menu_trans = single_menu.find('trans')
                if menu_trans:
                    menu_trans = menu_trans.get_text()
                menu_list.append({'menu_sense': menu_sense, 'menu_data': menu_data, 'menu_trans': menu_trans if menu_trans else None})
            word_dict['MENU'] = menu_list
        
        
        sense_list = []
        
        if entry.find('SENSE45') != None:
            
            for sense in entry.find_all('SENSE45'):

                sense45_dict = def_trans_ex(sense)

                sense_list.append(sense45_dict)



        if entry.find('SENSE') != None:
            if not sense_list:
                sense_list = []
            for sense in entry.find_all('SENSE'):

                sense_dict = def_trans_ex(sense)

  
                sense_list.append(sense_dict)

        phrase_list = []
        if entry.find('PHRASE') != None:

            for sense in entry.find_all('PHRASE'):
                phrase_dict = {}
                # DEFINITION
                if sense.find('DEFINITION') != None:
                    phrase_dict['DEFINITION'] = sense.find('DEFINITION').get_text()
                #MULTIWORD    
                if sense.find('MULTIWORD') != None:
                    phrase_dict['MULTIWORD'] = sense.find('MULTIWORD').get_text()    
                # TRANSLATION
                if sense.find('trans') != None:
                    phrase_dict['TRANSLATION'] = sense.find("trans").get_text()
                #EXAMPLES

                if sense.find('EXAMPLES') != None:
                    #examples_dic = {}
                    examples_dic = call_example(sense.find('EXAMPLES'))

                    phrase_dict['EXAMPLES'] = examples_dic
            phrase_list.append(phrase_dict)

        word_dict['ID'] = id if id else None

        if phrase_list:
            word_dict['PHRASE'] = phrase_list
        else:
            word_dict['PHRASE'] = None
        word_dict['SENSE'] = sense_list if sense_list else None


        word[vocab] = word_dict
    #f.write({vocab: word_dict})
    #yield {vocab: word_dict}
    #word_dict[] =
print(len(word))

# line 59

100%|███████████████████████████████████| 44309/44309 [00:29<00:00, 1520.50it/s]

44197


In [19]:
import pprint
# STYLE_LEVEL MENU
print(word['abandon1'])

{'PART-OF-SPEECH': 'verb', 'MENU': [{'menu_sense': '1', 'menu_data': 'leave sb', 'menu_trans': '拋下某人'}, {'menu_sense': '2', 'menu_data': 'leave a place', 'menu_trans': '離開某地'}, {'menu_sense': '3', 'menu_data': 'leave sth somewhere', 'menu_trans': '丟棄'}, {'menu_sense': '4', 'menu_data': 'stop doing/planning sth', 'menu_trans': '中止'}, {'menu_sense': '5', 'menu_data': 'stop supporting an idea', 'menu_trans': '放棄某個想法'}], 'ID': '000271', 'PHRASE': [{'DEFINITION': 'to feel an emotion so strongly that you do not think about anything else', 'MULTIWORD': 'abandon yourself to sth', 'TRANSLATION': '耽溺，陷入（某種情緒）'}], 'SENSE': [{'STYLE_LEVEL': 'informal', 'DEFINITION': 'to leave someone when you should stay with them and look after them', 'TRANSLATION': '拋棄；遺棄', 'EXAMPLES': {'EXAMPLE': 'His mother abandoned him when he was five days old.', 'TRANSLATION': '他出生才5天母親就遺棄了他。'}, 'SUB-SENSE': [{'STYLE_LEVEL': 'informal', 'DEFINITION': 'to stop supporting or helping someone', 'TRANSLATION': '背棄', 'EXAMPLES':

In [21]:
# GRAMMAR
word['central']

{'PART-OF-SPEECH': 'adj',
 'GRAMMAR': 'only before noun',
 'ID': '029891',
 'PHRASE': None,
 'SENSE': [{'DEFINITION': 'the central part of a place is the area in the middle of it',
   'TRANSLATION': '中心的；在中間的',
   'EXAMPLES': {'EXAMPLE': 'They live in central London.',
    'TRANSLATION': '他們住在倫敦市中心。'},
   'SUB-SENSE': [{'DEFINITION': 'a central place is in the middle of something',
     'TRANSLATION': '中央的',
     'EXAMPLES': {'EXAMPLE': 'The hotels rooms are built around a central courtyard.',
      'TRANSLATION': '旅館的房間是圍繞著中庭而建的。'}},
    {'DEFINITION': 'used for saying that something is in the middle of an area, so it is very convenient for using shops, theatres etc',
     'TRANSLATION': '位於中心地區的',
     'EXAMPLES': {'EXAMPLE': 'My flat is very central.',
      'TRANSLATION': '我住的公寓位於中心地帶。'}},
    {'DEFINITION': 'something that is central is extremely important, and often causes something else to happen or develop',
     'TRANSLATION': '極其重要的；起主導作用的'},
    {'DEFINITION': 'a central org

### Output example
`my_dict['record1']`
```json
{
    'ID': '284721',
    'PART-OF-SPEECH': 'noun',
    'PHRASE': [
               {'DEFINITION': '...',
                'EXAMPLES': ...,
                'MULTIWORD': '...',
                'TRANSLATION': '...'},
                ...
               ],
            
     'SENSE': [
               {'DEFINITION': '...',
                'EXAMPLES': ...,
                'SUB-SENSE': [{'DEFINITION': '...',
                               'EXAMPLES': ...,
                               'TRANSLATION': '...'}],
                'TRANSLATION': '...'},
                ...
               ]
}
```

In [22]:
word['divide1']

{'PART-OF-SPEECH': 'verb',
 'GRAMMAR': 'T',
 'MENU': [{'menu_sense': '1',
   'menu_data': 'separate/be separated',
   'menu_trans': '分開/分成'},
  {'menu_sense': '2', 'menu_data': 'be in between', 'menu_trans': '隔開'},
  {'menu_sense': '3', 'menu_data': 'in mathematics', 'menu_trans': '(數學)除,除以'},
  {'menu_sense': '4', 'menu_data': 'cause disagreement', 'menu_trans': '引起分歧'},
  {'menu_sense': '5',
   'menu_data': 'separate into two',
   'menu_trans': '(道路)分岔'},
  {'menu_sense': '6',
   'menu_data': 'when cells separate',
   'menu_trans': '(細胞)分裂'}],
 'ID': '101551',
 'PHRASE': [{'DEFINITION': 'to do one thing or be in one place for some of the time, and do another thing or be in another place for the rest of the time',
   'MULTIWORD': 'divide your time between',
   'TRANSLATION': '在…之間分配時間',
   'EXAMPLES': {'EXAMPLE': 'She divides her time between teaching and research.',
    'TRANSLATION': '她把時間分配在教學和研究兩方面。'}}],
 'SENSE': [{'DEFINITION': 'to separate people or things into smaller groups o

In [20]:
word['reference1']

{'PART-OF-SPEECH': 'noun',
 'GRAMMAR': 'C/U',
 'MENU': [{'menu_sense': '1',
   'menu_data': 'mention of sb/sth',
   'menu_trans': '提到'},
  {'menu_sense': '2',
   'menu_data': 'looking for information',
   'menu_trans': '查閱'},
  {'menu_sense': '3', 'menu_data': 'statement about you', 'menu_trans': '介紹信'},
  {'menu_sense': '4',
   'menu_data': 'number for finding sth',
   'menu_trans': '標號;代號'},
  {'menu_sense': '5',
   'menu_data': 'phrase from book/poem',
   'menu_trans': '引文'},
  {'menu_sense': '6',
   'menu_data': 'a mention in writing',
   'menu_trans': '參考書目'}],
 'ID': '285871',
 'PHRASE': [{'DEFINITION': 'used in a letter to introduce the subject that you are going to write about',
   'MULTIWORD': 'with reference to',
   'TRANSLATION': '（用於信函中）關於'}],
 'SENSE': [{'STYLE_LEVEL': 'formal',
   'DEFINITION': 'a comment that mentions something or someone',
   'TRANSLATION': '提及，提到',
   'EXAMPLES': {'EXAMPLE': 'The title is apparently a reference to the singers brother.',
    'TRANSLATIO